In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install librosa

In [ ]:
!pip install soundfile


In [ ]:
import os
import librosa
import numpy as np



In [ ]:
import os
import pandas as pd

def read_emotion_dataset(dataset_folder):
    data = {'file_path': [], 'emotion': []}

    for emotion in os.listdir(dataset_folder):
        emotion_folder = os.path.join(dataset_folder, emotion)

        if os.path.isdir(emotion_folder):

            for file_name in os.listdir(emotion_folder):
                file_path = os.path.join(emotion_folder, file_name)

                if os.path.isfile(file_path):

                    data['file_path'].append(file_path)
                    data['emotion'].append(emotion)

    emotion_dataset = pd.DataFrame(data)
    return emotion_dataset

dataset_folder = "/content/drive/MyDrive/NLP/iiitproject"
emotion_dataset = read_emotion_dataset(dataset_folder)

print(emotion_dataset.tail(50))


                                              file_path  emotion
6305  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6306  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6307  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6308  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6309  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6310  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6311  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6312  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6313  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6314  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6315  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6316  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6317  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6318  /content/drive/MyDrive/NLP/iiitproject/disgust...  disgust
6319  /content/drive/MyDr

In [ ]:
!pip install soundfile sounddevice


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
emotion_df=emotion_dataset

In [ ]:
!pip install librosa
!pip install resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.6 MB/s eta 0:00:00


In [ ]:
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder

def extract_features(file_path):
    audio, _ = librosa.load(file_path, sr=None)
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=_ , n_mfcc=13), axis=1)
    return mfccs
X = [extract_features(file_path) for file_path in emotion_df['file_path']]
X = np.vstack(X)
y = emotion_df['emotion']

le = LabelEncoder()
y = le.fit_transform(y)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4257, 13)
(4257,)
(2098, 13)
(2098,)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
mlp = MLPClassifier(hidden_layer_sizes=(50, 50), activation='relu', solver='adam',
                    alpha=0.0001, batch_size='auto', learning_rate='constant',
                    learning_rate_init=0.001, max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=500, random_state=42)

In [ ]:
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")


Accuracy: 43.57%


In [ ]:
import pickle
train_test_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test
}

# Save the data to a pickle file
with open('train_test_data.pkl', 'wb') as file:
    pickle.dump(train_test_data, file)
with open('your_label_encoder_file.pkl', 'wb') as label_encoder_file:
    pickle.dump(le, label_encoder_file)


In [ ]:
import librosa
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier

with open('train_test_data.pkl', 'rb') as model_file:
    model_dict = pickle.load(model_file)

print(model_dict.keys())


emotion_labels = ['anger', 'disgust', 'fear','sad','surprise']  # Replace with your actual emotion labels

def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None, res_type='kaiser_fast')
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13), axis=1)
    return mfccs

new_audio_file_path = '/content/drive/MyDrive/NLP/hello.mp3'

new_audio_features = extract_features(new_audio_file_path)
new_audio_features = new_audio_features.reshape(1, -1)
print(new_audio_features)
emotion_pred = mlp.predict(new_audio_features)
print(emotion_pred)
predicted_emotion_label = le.inverse_transform(emotion_pred)[0]
print(predicted_emotion_label)
predicted_emotion_name = emotion_labels[predicted_emotion_label]

print(f"Predicted Emotion: {predicted_emotion_label} ({predicted_emotion_name})")


dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
[[-5.3521094e+02  1.1412729e+02  1.9520229e+01  1.5574841e+01
  -7.6813488e+00  2.5339651e+01 -2.0562569e+01  1.9316656e+01
  -1.3230029e+01  7.6450366e-01 -7.4098911e+00 -3.2325075e+00
   1.1329950e-01]]
[1]
1
Predicted Emotion: 1 (disgust)


In [ ]:
print("Keys in the dictionary:", model_dict.keys())

Keys in the dictionary: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization,Dropout
# Define the model architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(emotion_labels), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('emotion_model_updated.h5')


Epoch 1/50
67/67 [==============================] - 3s 11ms/step - loss: 1.9545 - accuracy: 0.3531 - val_loss: 1.3951 - val_accuracy: 0.4290
Epoch 2/50
67/67 [==============================] - 0s 5ms/step - loss: 1.6706 - accuracy: 0.3963 - val_loss: 1.3831 - val_accuracy: 0.3851
Epoch 3/50
67/67 [==============================] - 1s 8ms/step - loss: 1.4702 - accuracy: 0.4299 - val_loss: 1.3495 - val_accuracy: 0.4199
Epoch 4/50
67/67 [==============================] - 1s 10ms/step - loss: 1.4112 - accuracy: 0.4461 - val_loss: 1.2836 - val_accuracy: 0.4647
Epoch 5/50
67/67 [==============================] - 1s 8ms/step - loss: 1.3612 - accuracy: 0.4489 - val_loss: 1.2674 - val_accuracy: 0.4757
Epoch 6/50
67/67 [==============================] - 0s 5ms/step - loss: 1.3177 - accuracy: 0.4517 - val_loss: 1.2585 - val_accuracy: 0.4762
Epoch 7/50
67/67 [==============================] - 0s 5ms/step - loss: 1.3091 - accuracy: 0.4496 - val_loss: 1.2260 - val_accuracy: 0.4781
Epoch 8/50
67/67 [

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
